# Download Data

## https://data.police.uk/data/

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd

In [2]:
uri = "bolt://localhost:7687/neo4j"  # Replace with your Neo4j server URI
username = "your_username"
password = "your_password"

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

## Define ingest function

In [5]:
csv_directory = "D:/dataset/police-uk-crime"

def create_indexes(tx):
    tx.run("CREATE INDEX FOR (y:Year) ON (y.value)")
    tx.run("CREATE INDEX FOR (m:Month) ON (m.value)")
    tx.run("CREATE INDEX FOR (p:Place) ON (p.name)")
    tx.run("CREATE INDEX FOR (ctg:Category) ON (ctg.name)")

#for local to local
def load_data_from_csv_cypher(tx, file_path):
    tx.run(f"""
        LOAD CSV WITH HEADERS FROM "{file_path}" AS csv
        WITH csv AS crimecsv
        WHERE crimecsv.`Location` IS NOT NULL
        MERGE (p:Place {{name: crimecsv.`Location`}})
        WITH crimecsv, split(crimecsv.`Month`, "-") AS yearMonth
        WHERE yearMonth[0] IS NOT NULL
        MERGE (y:Year {{value: toInteger(yearMonth[0])}})
        WITH crimecsv, split(crimecsv.`Month`, "-") AS yearMonth
        WHERE yearMonth[1] IS NOT NULL
        MERGE (m:Month {{value: toInteger(yearMonth[1])}})
        WITH crimecsv
        WHERE crimecsv.`Crime type` IS NOT NULL
        MERGE (ctg:Category {{name: crimecsv.`Crime type`}})
        WITH crimecsv
        WHERE crimecsv.`Latitude` IS NOT NULL
        MERGE (c:Case {{
            ref: CASE WHEN crimecsv.`Crime ID` IS NULL THEN '' ELSE crimecsv.`Crime ID` END,
            lat: toFloat(crimecsv.`Latitude`),
            lon: toFloat(crimecsv.`Longitude`),
            outcome: CASE WHEN crimecsv.`Last outcome category` IS NULL THEN 'n/a' ELSE crimecsv.`Last outcome category` END
        }})
        WITH crimecsv, split(crimecsv.`Month`, "-") AS yearMonth
        MATCH (xc:Case {{
            ref: CASE WHEN crimecsv.`Crime ID` IS NULL THEN '' ELSE crimecsv.`Crime ID` END,
            lat: toFloat(crimecsv.`Latitude`),
            lon: toFloat(crimecsv.`Longitude`),
            outcome: CASE WHEN crimecsv.`Last outcome category` IS NULL THEN 'n/a' ELSE crimecsv.`Last outcome category` END
        }}),
        (xy:Year {{value: toInteger(yearMonth[0])}}),
        (xm:Month {{value: toInteger(yearMonth[1])}}),
        (xp:Place {{name: crimecsv.`Location`}}),
        (xctg:Category {{name: crimecsv.`Crime type`}})
        CREATE (xm)-[:YEAR_OF]->(xy),
        (xc)-[:HAPPEN_IN]->(xm),
        (xc)-[:TYPE_OF]->(xctg),
        (xc)-[:AT]->(xp)
    """)

#for local to docker without copy files
def load_data_from_csv(tx, df):
    for index, row in df.iterrows():
        location = row['Location']
        year, month = map(int, row['Month'].split('-'))
        crime_type = row['Crime type']
        latitude = row['Latitude']
        longitude = row['Longitude']
        crime_id = str(row['Crime ID']) if not pd.isnull(row['Crime ID']) else ''
        outcome_category = str(row['Last outcome category']) if not pd.isnull(row['Last outcome category']) else 'n/a'
        force = row['Reported by']
        # Merge or create nodes based on your data model
        tx.run("""
            MERGE (p:Place {name: $location})
            MERGE (f:Force {name: $force})

            MERGE (m:Month {value: $month})
            MERGE (ctg:Category {name: $crime_type})
            MERGE (c:Case {
                ref: $crime_id,
                lat: $latitude,
                lon: $longitude,
                outcome: $outcome_category
            })
            WITH c,  m, p, ctg,f
            CREATE 
                   (c)-[:REPORTED_BY]->(f),
                   (c)-[:HAPPEN_IN]->(m),
                   (c)-[:TYPE_OF]->(ctg),
                   (c)-[:AT]->(p)
        """, location=location, month=month, crime_type=crime_type,force=force,
             latitude=latitude, longitude=longitude, crime_id=crime_id, outcome_category=outcome_category)
        
def create_time(tx,year,month):
        tx.run("""
            MERGE (y:Year {value: $year})
            MERGE (m:Month {value: $month})
   
            WITH y, m
            CREATE (m)-[:YEAR_OF]->(y)
        """, year=year, month=month)

def create_police_force(tx,force):
        tx.run("""
            MERGE (f:Force {name: $force})

        """, force=force)


## Data preparation

In [6]:
# with driver.session() as session:
#     session.write_transaction(create_indexes)
df_list = []
# Loop through files in the directory
for root, dirs, files in os.walk(csv_directory):
    for file in files:
        # print(file)
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # print(file_path)
            # Execute the load data query for each file
            # with driver.session() as session:
            #     session.write_transaction(load_data_from_csv_cypher, file_path)
            df = pd.read_csv(file_path)
            df['Latitude'].fillna(0, inplace=True)
            df['Longitude'].fillna(0, inplace=True)
            df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
            df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
            df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
            df['Location'] = df['Location'].str.strip()
            df.drop_duplicates(inplace=True)
            df_list.append(df)

            # Execute the load data query for each file
            # with driver.session() as session:
            #     session.execute_write(load_data_from_csv, df)

## Create month and year

In [36]:
for dir in os.listdir(csv_directory):
    # print(dir)
    [month,year] = dir.split('-')
    print(month,year)
    with driver.session() as session:
        session.execute_write(create_time, year,month)

2022 06
2022 07
2022 08
2022 09
2022 10
2022 11
2022 12
2023 01
2023 02
2023 03
2023 04
2023 05
2023 06
2023 07
2023 08
2023 09
2023 10
2023 11
2023 12
2024 01


In [40]:
full_df = pd.concat(df_list,axis=0)

## Ingest police force

In [44]:
full_df[full_df['Reported by']!=full_df['Falls within']]#.unique()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context


In [46]:
for force in  full_df['Reported by'].unique():
    with driver.session() as session:
        session.execute_write(create_police_force, force)

## Ingest crimes and create relations

In [7]:
for df in df_list:
    with driver.session() as session:
        session.execute_write(load_data_from_csv, df)